In [1]:
import json
import pandas as pd
from keplergl import KeplerGl

In [2]:
geojson_file = 'Russia_regions.geojson'

In [3]:
with open(geojson_file) as data_file:
    data = json.load(data_file)
geodata = json.loads(json.dumps(data))

In [4]:
chuk = next(item for item in geodata['features'] if item["properties"]["region"] == "Чукотский АО")
for i in chuk['geometry']['coordinates']:
    for j in i[0]:
        if j[0] < 0:
            j[0] = 360 + j[0]
index = next(i for i in range(len(geodata['features'])) if geodata['features'][i]["properties"]["region"] == "Чукотский АО")
geodata['features'][index] = chuk

In [5]:
map_1 = KeplerGl(height=500)

User Guide: https://docs.kepler.gl/docs/keplergl-jupyter


In [6]:
regions_df = pd.read_csv("regions.csv", sep=";")
regions_df = regions_df.drop(["Unnamed: 3"], axis=1)
regions = {}
for idx, region in regions_df.iterrows():
    regions[region['region']] = region['code']
    regions[region['code']] = region['region']
regions_df.set_index('code', inplace=True)

In [7]:
cprm_df = pd.read_csv("counts_by_regions_and_years.csv", sep=";")
cprm_df.set_index('regions', inplace=True)

In [8]:
for feature in geodata['features']:
    try:
        cases_by_year_in_region = cprm_df[regions[feature["properties"]["region"]]]
        ppl_count_mln = float(regions_df['mln']['RU-ALT'].replace(',', '.'))
        feature["properties"]["2020"] = str(cases_by_year_in_region["2020"]/ppl_count_mln)
        feature["properties"]["2021"] = str(cases_by_year_in_region["2021"]/ppl_count_mln)
        feature["properties"]["2022"] = str(cases_by_year_in_region["2022"]/ppl_count_mln)
        feature["properties"]["2023"] = str(cases_by_year_in_region["2023"]/ppl_count_mln)
        feature["properties"]["total"] = str(cases_by_year_in_region["total"]/ppl_count_mln)
    except Exception:
        feature["properties"]["2020"] = "0"
        feature["properties"]["2021"] = "0"
        feature["properties"]["2022"] = "0"
        feature["properties"]["2023"] = "0"
        feature["properties"]["total"] = "0"

In [ ]:
map_1.add_data(data=geodata, name="geojson")
map_1.save_to_html(file_name="map.html")

Map saved to map.html!
